In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist', split=['train', 'test'], as_supervised=True, with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.71X6ZA_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.71X6ZA_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalisation entre 0 et 1
    return tf.reshape(image, [-1]), label       # Aplatir l'image

In [ ]:
ds_train = ds_train.map(preprocess).batch(64)
ds_test = ds_test.map(preprocess).batch(64)

In [ ]:
W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1))
b1 = tf.Variable(tf.zeros([128]))
W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

In [ ]:
def model(x):
    x = tf.nn.relu(tf.matmul(x, W1) + b1)  # Couche 1 + ReLU
    return tf.matmul(x, W2) + b2           # Couche 2 (logits, sans softmax)

In [ ]:
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(0.001)

In [ ]:
for epoch in range(5):
    for x_batch, y_batch in ds_train:
        with tf.GradientTape() as tape:
            logits = model(x_batch)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, [W1, b1, W2, b2])
        optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2]))
    print(f"Epoch {epoch+1} terminé")

Epoch 1 terminé
Epoch 2 terminé
Epoch 3 terminé
Epoch 4 terminé
Epoch 5 terminé


In [ ]:
correct = 0
total = 0
for x_batch, y_batch in ds_test:
    logits = model(x_batch)
    preds = tf.argmax(logits, axis=1, output_type=tf.int32)
    y_batch = tf.cast(y_batch, tf.int32)
    correct += tf.reduce_sum(tf.cast(preds == y_batch, tf.int32))
    total += x_batch.shape[0]
print("Accuracy :", correct.numpy() / total)

Accuracy : 0.9744
